In [1]:
from sqlalchemy import Column, DateTime, Integer, String, Boolean, ForeignKey, create_engine, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker, relationship
import os
if os.path.exists("orm-test.db"):
    os.remove("orm-test.db")
engine = create_engine('sqlite:///orm-test.db', echo=True, convert_unicode=True)
Base = declarative_base()

In [2]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    username = Column(String(31), unique=True, nullable=False)
    password = Column(String(31))
    name = Column(String(255))
    email = Column(String(255))
    phone = Column(String(255))
    admin = Column(Boolean, default=False)
    investor = Column(Boolean, default=False)
    locale = Column(String(5), default='en_US')
    verified = Column(Boolean, default=False)
    blocked = Column(Boolean, default=False)
    created = Column(DateTime(), server_default=text("(datetime('now'))"))
    updated = Column(DateTime(), server_default=text("(datetime('now'))"))
    bids = relationship("Bid", back_populates="bidder")
    asks = relationship("Ask", back_populates="investor")

    def __repr__(self):
        return "<User(id='%s', username='%s', email='%s', phone='%s', admin='%s', verified='%s', blocked='%s')>" % (
            self.id, self.username, self.email, self.phone, self.admin, self.verified, self.blocked
        )

    def update(self, **kwargs):
        if len(kwargs):
            p = ('username', 'password', 'name', 'email', 'phone', 'admin', 'investor', 'locale', 'verified', 'blocked')
            for k, v in kwargs.items():
                if k in p:
                    self.__setattr__(k, v)

    def dump(self):
        return dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])


class Bid(Base):
    __tablename__ = 'bids'
    id = Column(Integer, primary_key=True)
    quantity = Column(Integer)
    price = Column(Integer)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    status = Column(Boolean, default=True)
    created = Column(DateTime(), server_default=text("(datetime('now'))"))
    updated = Column(DateTime(), server_default=text("(datetime('now'))"))
    bidder = relationship("User", back_populates="bids")

    def __repr__(self):
        return "<Bid(id='%s', quantity='%s', price='%s', user_id='%s', status='%s')>" % (
            self.id, self.quantity, self.price, self.user_id, self.status
        )

    def update(self, **kwargs):
        if len(kwargs):
            p = ('quantity', 'price', 'bidder', 'status')
            for k, v in kwargs.items():
                if k in p:
                    self.__setattr__(k, v)

    def dump(self):
        return dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])

class Ask(Base):
    __tablename__ = 'asks'
    id = Column(Integer, primary_key=True)
    quantity = Column(Integer)
    price = Column(Integer)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    status = Column(Boolean, default=True)
    created = Column(DateTime(), server_default=text("(datetime('now'))"))
    updated = Column(DateTime(), server_default=text("(datetime('now'))"))
    investor = relationship("User", back_populates="asks")

    def __repr__(self):
        return "<Ask(id='%s', quantity='%s', price='%s', user_id='%s', status='%s')>" % (
            self.id, self.quantity, self.price, self.user_id, self.status
        )

    def update(self, **kwargs):
        if len(kwargs):
            p = ('quantity', 'price', 'bidder', 'status')
            for k, v in kwargs.items():
                if k in p:
                    self.__setattr__(k, v)

    def dump(self):
        return dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])

In [3]:
db_session = scoped_session(sessionmaker(autocommit=False, autoflush=False, bind=engine))

In [4]:
Base.query = db_session.query_property()

In [5]:
Base.metadata.create_all(bind=engine)

2018-12-13 16:18:54,612 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-12-13 16:18:54,613 INFO sqlalchemy.engine.base.Engine ()
2018-12-13 16:18:54,614 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-12-13 16:18:54,615 INFO sqlalchemy.engine.base.Engine ()
2018-12-13 16:18:54,617 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-12-13 16:18:54,618 INFO sqlalchemy.engine.base.Engine ()
2018-12-13 16:18:54,619 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("bids")
2018-12-13 16:18:54,620 INFO sqlalchemy.engine.base.Engine ()
2018-12-13 16:18:54,621 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("asks")
2018-12-13 16:18:54,621 INFO sqlalchemy.engine.base.Engine ()
2018-12-13 16:18:54,623 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	username VARCHAR(31) NOT NULL, 
	password VARCHAR(31), 
	name VARCHAR(255), 
	email VARCHAR

In [6]:
u1 = User(username='pericaperic')

In [7]:
db_session.query(User).all()

2018-12-13 16:18:59,916 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-13 16:18:59,918 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password, users.name AS users_name, users.email AS users_email, users.phone AS users_phone, users.admin AS users_admin, users.investor AS users_investor, users.locale AS users_locale, users.verified AS users_verified, users.blocked AS users_blocked, users.created AS users_created, users.updated AS users_updated 
FROM users
2018-12-13 16:18:59,919 INFO sqlalchemy.engine.base.Engine ()


[]

In [8]:
db_session.add(u1)

In [9]:
db_session.commit()

2018-12-13 16:19:03,057 INFO sqlalchemy.engine.base.Engine INSERT INTO users (username, password, name, email, phone, admin, investor, locale, verified, blocked) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2018-12-13 16:19:03,058 INFO sqlalchemy.engine.base.Engine ('pericaperic', None, None, None, None, 0, 0, 'en_US', 0, 0)
2018-12-13 16:19:03,060 INFO sqlalchemy.engine.base.Engine COMMIT


In [10]:
u1.bids.append(Bid(quantity=3,price=3456))

2018-12-13 16:19:03,819 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-13 16:19:03,820 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password, users.name AS users_name, users.email AS users_email, users.phone AS users_phone, users.admin AS users_admin, users.investor AS users_investor, users.locale AS users_locale, users.verified AS users_verified, users.blocked AS users_blocked, users.created AS users_created, users.updated AS users_updated 
FROM users 
WHERE users.id = ?
2018-12-13 16:19:03,821 INFO sqlalchemy.engine.base.Engine (1,)
2018-12-13 16:19:03,823 INFO sqlalchemy.engine.base.Engine SELECT bids.id AS bids_id, bids.quantity AS bids_quantity, bids.price AS bids_price, bids.user_id AS bids_user_id, bids.status AS bids_status, bids.created AS bids_created, bids.updated AS bids_updated 
FROM bids 
WHERE ? = bids.user_id
2018-12-13 16:19:03,824 INFO sqlalchemy.engine.base.Engine (1,)


In [11]:
db_session.new

IdentitySet([<Bid(id='None', quantity='3', price='3456', user_id='None', status='None')>])

In [12]:
db_session.commit()

2018-12-13 16:19:06,523 INFO sqlalchemy.engine.base.Engine INSERT INTO bids (quantity, price, user_id, status) VALUES (?, ?, ?, ?)
2018-12-13 16:19:06,524 INFO sqlalchemy.engine.base.Engine (3, 3456, 1, 1)
2018-12-13 16:19:06,526 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
u1.bids

2018-12-13 16:19:06,994 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-13 16:19:06,995 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password, users.name AS users_name, users.email AS users_email, users.phone AS users_phone, users.admin AS users_admin, users.investor AS users_investor, users.locale AS users_locale, users.verified AS users_verified, users.blocked AS users_blocked, users.created AS users_created, users.updated AS users_updated 
FROM users 
WHERE users.id = ?
2018-12-13 16:19:06,995 INFO sqlalchemy.engine.base.Engine (1,)
2018-12-13 16:19:06,997 INFO sqlalchemy.engine.base.Engine SELECT bids.id AS bids_id, bids.quantity AS bids_quantity, bids.price AS bids_price, bids.user_id AS bids_user_id, bids.status AS bids_status, bids.created AS bids_created, bids.updated AS bids_updated 
FROM bids 
WHERE ? = bids.user_id
2018-12-13 16:19:06,999 INFO sqlalchemy.engine.base.Engine (1,)


[<Bid(id='1', quantity='3', price='3456', user_id='1', status='True')>]

In [14]:
u1.asks.append(Ask(quantity=3,price=3456))

2018-12-13 16:19:08,989 INFO sqlalchemy.engine.base.Engine SELECT asks.id AS asks_id, asks.quantity AS asks_quantity, asks.price AS asks_price, asks.user_id AS asks_user_id, asks.status AS asks_status, asks.created AS asks_created, asks.updated AS asks_updated 
FROM asks 
WHERE ? = asks.user_id
2018-12-13 16:19:08,990 INFO sqlalchemy.engine.base.Engine (1,)


In [15]:
db_session.commit()

2018-12-13 16:19:09,637 INFO sqlalchemy.engine.base.Engine INSERT INTO asks (quantity, price, user_id, status) VALUES (?, ?, ?, ?)
2018-12-13 16:19:09,639 INFO sqlalchemy.engine.base.Engine (3, 3456, 1, 1)
2018-12-13 16:19:09,640 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
u1.asks

2018-12-13 16:19:12,826 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-13 16:19:12,828 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password, users.name AS users_name, users.email AS users_email, users.phone AS users_phone, users.admin AS users_admin, users.investor AS users_investor, users.locale AS users_locale, users.verified AS users_verified, users.blocked AS users_blocked, users.created AS users_created, users.updated AS users_updated 
FROM users 
WHERE users.id = ?
2018-12-13 16:19:12,829 INFO sqlalchemy.engine.base.Engine (1,)
2018-12-13 16:19:12,831 INFO sqlalchemy.engine.base.Engine SELECT asks.id AS asks_id, asks.quantity AS asks_quantity, asks.price AS asks_price, asks.user_id AS asks_user_id, asks.status AS asks_status, asks.created AS asks_created, asks.updated AS asks_updated 
FROM asks 
WHERE ? = asks.user_id
2018-12-13 16:19:12,832 INFO sqlalchemy.engine.base.Engine (1,)


[<Ask(id='1', quantity='3', price='3456', user_id='1', status='True')>]